In [ ]:
import numpy as np
import pandas as pd

from parser import get_parser
from dataset import dataset_generator
from cre import CRE
from ite import estimate_ite

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

# Simulations

In [ ]:
# settings
N = 2000
t_ss = 0.9
effect_size = 5
n_seeds = 10
methods = ["tlearner", "slearner", "xlearner","causalforest", "drlearner", "aipw"]

In [ ]:
result = pd.DataFrame(columns = ['estimator','seed','bias', 'RMSE'])
for method in tqdm(methods):
    for seed in range(n_seeds):
        X, y, z, ite = dataset_generator(n = N, 
                                         k = 10, 
                                         binary_cov = True,
                                         binary_out = False, 
                                         effect_size = effect_size,
                                         confounding = "no",
                                         n_rules = 2,
                                         seed = seed)
        args = get_parser().parse_args(args=[])
        args.verbose = False
        args.method = method
        args.t_ss = t_ss
        args.seed = seed

        model = CRE(args)
        model.fit(X, y, z)
        ite_pred = model.eval(X)
        bias = np.mean(ite - ite_pred)
        RMSE = np.sqrt(np.mean((ite - ite_pred)**2))
        result = result.append({'estimator': "CRE ("+method+")",
                                'seed': seed, 
                                'bias': bias, 
                                'RMSE': RMSE}, 
                                ignore_index=True)

        ite_pred = estimate_ite(X, y, z,
                                method = method,
                                learner_y = args.learner_y,
                                learner_ps = args.learner_ps)
        bias = np.mean(np.abs(ite - ite_pred))
        RMSE = np.sqrt(np.mean((ite - ite_pred)**2))
        result = result.append({'estimator': method,
                                'seed': seed, 
                                'bias': bias, 
                                'RMSE': RMSE}, 
                                ignore_index=True)
result

In [ ]:
# group by estimator
result.groupby('estimator').mean()